### Описание задачи

Сформировать набор товаров (до 25 штук), для пользователей, которые совершат покупки в следующие 3 дня.

Исходя из данных о поведении посетителей на сайте интернет-магазина, необходимо разработать модель для рекомендации товаров пользователям. Модель должна предсказывать топ 25 товаров, которые пользователь купит с наибольшей вероятностью в следующие 3 дня (период тестового датасета). 
Метрикой качества является Recall по купленным товарам. Пользователи, не купившие ни одного товара, в расчете метрики не учитываются.

Например, в сформированном вашей моделью списке из 25 товаров, оказалось 3 из 4х, которые пользователь реально купил. Recall, таким образом, составит ¾ = 0.75. Если пользователь ничего не купил, recall для него не считается.

При этом, топ товаров составляется для пользователей, которые с наибольшей вероятностью совершат покупку. По топу пользователей рассчитывается отдельная метрика - Precision по факту покупки хотя бы одного товара.

In [1]:
import numpy as np
from datetime import datetime
import pandas as pd

#### Считываем данны о товарах

In [3]:
catalog = pd.read_parquet('stokman_catalog_preprocessed.pq', engine='pyarrow')
catalog.head()

,add_date,shop_id,product_id,category_id,price,title,old_price
0,2024-09-30 03:07:09,350,1025536,8922,6990,6048 48557 44342 53515 17614,6990
1,2024-09-19 03:07:08,350,1025614,10600,330,42548 40244 23485 10853 6545 2343,330
2,2024-09-30 03:07:09,350,1025656,231,13240,14034 42002 48557 56089 46836 55620 50785,26490
3,2024-09-30 03:07:09,350,1025659,14546,7550,32640 50164 48557 25314 23255 42146,12590
4,2024-09-30 03:07:09,350,1025668,1949,1290,51789 48557 56089 20149,1290


#### Расшифровка признаков:
    add_date - дата добавления товара в магазин
    shop_id - идентификатор магазина
    product_id - идентификатор товара
    category_id - идентификатор категории товаров
    price - цена
    title - закодированное название товара (в файле vectors.npz содержатся эмбединги оригинальных названий)
    old_price - старая цена

#### Считываем тренировочные данные о действиях пользователей (период 3 недели)

In [7]:
actions= pd.read_parquet('train_actions.pq', engine='pyarrow')
actions = actions.sort_values(by='date')
actions.head()

,user_id,loc_user_id,action,date,products,pageId
0,6fd49b56-8cc6-11ed-86e0-002590c0647c,c6e357dc-121d-449d-a744-e9a0b56c2380,7,2024-09-07 00:00:04,[],2571824865
1,6fd49b56-8cc6-11ed-86e0-002590c0647c,c6e357dc-121d-449d-a744-e9a0b56c2380,7,2024-09-07 00:00:08,[],3834364438
2,f9c498ec-5d3b-11ef-86e0-002590c0647c,120c9064-1131-4dc3-8048-44184531b42e,7,2024-09-07 00:00:08,[],2448628415
3,59386b5c-e64f-11ec-8086-002590c82437,f0745572-893f-4e50-bc52-5af47badff5a,7,2024-09-07 00:00:11,[],3875013967
4,badbd396-6cab-11ef-86e0-002590c0647c,73423d85-d47c-4332-8155-5200615302b5,7,2024-09-07 00:00:11,[],3025531174


#### Расшифровка признаков:
    user_id - глобальный идентификатор пользователя для подбора рекомендаций (формируется по cookies. Один пользователь может заходить с разных сайтов или устройств и иметь разные ID) 
    loc_user_id - локальный идентификатор пользователя внутри сайта
    action - код действия пользователя
    date - дата и время действия
    products - массив идентификаторов товаров, относящийся к действию
    pageId - идентификатор страницы веб-сайта, с которой пользователь выполняет действие

#### Признак action в train_actions.pq имеет следующую расшифровку:
    0 - view (просмотр товара)
    1 - like (лайк товара)
    2 - addB (добавление товара в корзину)
    3 - delB (удаление товара из корзины)
    4 - clearB (удаление всех товаров из корзины)
    5 - order (оформление заказа)
    6 - listB (посещение страницы корзины и вывод списка товаров в корзине)
    7 - visit (посещение страницы с товаром)
    8 - visitCategory (посещение страницы с группой товаров)
    9 - search (поиск товара)

#### Формируем простую модель для прогноза пользователей, которые осуществят покупки в следующие 3 дня: 
Выбираем всех пользователей, которые добавляли товары в корзину (action=2) за последние 5 дней трейнировочной выборки и, в качестве рекомендаций, оставляем айтемы, которые добавлялись в корзину


In [13]:
def processing(purchasers):
    purchasers = purchasers.map(lambda x: x[0])
    purchasers = list(purchasers)
    return purchasers[:25]
    
# ищем дату: 5 дней до конца периода тренировочной выборки
last_5_days = actions['date'].max() - pd.Timedelta(5, unit='D') 
# выбираем все события добавления товара в корзину за последние 5 дней
purchasers = actions[(actions['action'] == 2) & (actions['date'] > last_5_days)]
# группируем выборку по уникальным пользователям, и сохраняем, в качестве рекомендаций товары (до 25), которые пользователь добавлял в корзину
purchasers = purchasers.groupby(['user_id'])['products'].apply(processing)
# сохраняем файл с предсказаниями для сабмита на платформу
purchasers = purchasers.reset_index().to_csv("predictions.csv", index=False)

In [15]:
predictions = pd.read_csv('predictions.csv')
predictions 

,user_id,products
0,0000bdba-5180-11eb-8a53-0cc47a6d2fef,['5168833']
1,000576f4-79eb-11ec-a6e9-002590c82437,['1806876']
2,0038911e-79a3-11ef-9b7b-002590c82436,['2745244']
3,0048f02c-75c1-11e9-8a53-0cc47a6d2fef,['1774419']
4,004b41f8-6098-11ee-86e0-002590c0647c,['6596864']
...,...,...
3593,ffbe348a-9f39-11ee-86e0-002590c0647c,"['4811953', '4040418']"
3594,ffdcd80a-82c8-11e9-a0d7-002590e45c38,"['722947', '6151246', '6151264', '1854850', '1..."
3595,ffe12968-5037-11ef-9b7b-002590c82436,['7073931']
3596,ffeeade6-1fee-11ea-86e0-002590c0647c,['6640276']
